In [40]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# from substitutions import appos
from nltk.stem import PorterStemmer, WordNetLemmatizer
import copy
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import string
import numpy as np
import contractions
from nltk import RegexpTokenizer


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sarthakj01/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sarthakj01/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/sarthakj01/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/sarthakj01/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [41]:
data_dir = '../Humor,Hist,Media,Food'
file_names = os.listdir(data_dir)
file_paths = [(data_dir + '/' + fname) for fname in file_names]
docID_to_doc = {}
for i in range(len(file_names)):
    docID_to_doc[i] = file_names[i]

In [42]:
def remove_punct(tok):
    punctuations = string.punctuation
    tok = ''.join(x for x in tok if x not in punctuations)
    return tok
def remove_space(tok):
    tok = ''.join(x for x in tok if x != ' ')
    return tok

def preprocess_file(file_text):

    ftext = file_text.lower()
    # print(f"Original Text : \n{ftext}\n\n")
    # print("\n---------------------------------------\n")
    file_toks = word_tokenize(ftext)
    # print(f"After word_Tokenization : \n{file_toks}\n\n")
    # print("---------------------------------------")
    stop_words = list(set(stopwords.words('english')))
    file_toks = [tok for tok in file_toks if tok not in stop_words]
    # print(f"After removing stopwords : \n{file_toks}\n\n")
    # print("\n---------------------------------------\n")
    toks_no_punct = []
    for tok in file_toks:
        ctok = remove_punct(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            toks_no_punct.append(ctok)
    # print("\n---------------------------------------\n")
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            cleaned_toks.append(ctok)
    return cleaned_toks

def cleanQuery(file_text):

    ftext = file_text.lower()
    # print(f"Original Text : \n{ftext}\n\n")
    # print("\n---------------------------------------\n")
    file_toks = word_tokenize(ftext)
    # print(f"After word_Tokenization : \n{file_toks}\n\n")
    # print("---------------------------------------")
    stop_words = list(set(stopwords.words('english')))
    file_toks = [tok for tok in file_toks if tok not in stop_words]
    # print(f"After removing stopwords : \n{file_toks}\n\n")
    # print("\n---------------------------------------\n")
    toks_no_punct = []
    for tok in file_toks:
        ctok = remove_punct(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            toks_no_punct.append(ctok)
    # print("\n---------------------------------------\n")
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            cleaned_toks.append(ctok)
    return cleaned_toks

In [43]:
def read_file(fpaths):
    
    file_tokens = []
    for fpath in fpaths:
        # print(fpath)
        f = open(fpath, 'r', encoding='utf-8', errors='ignore')
        ftxt_unprocessed = f.read()
        ftoks = preprocess_file(ftxt_unprocessed)
        file_tokens.append(ftoks)
    return file_tokens


In [44]:
def getDocsFromID(file_names, doc_IDs):
    doc_names = []
    for doc_ID in doc_IDs:
        doc_names.append(file_names[doc_ID])
    return doc_names

In [45]:
file_toks = read_file(file_paths)

In [46]:
def create_positional_index(file_toks):
    pidx_dict = {}
    for i in range(len(file_toks)):
        for j in range(len(file_toks[i])):
            tok = file_toks[i][j]
            if(tok not in pidx_dict.keys()):
                pidx_dict[tok] = {}
                pidx_dict[tok][i] = [j]
            else:
                if(i in pidx_dict[tok].keys()):
                    pidx_dict[tok][i].append(j)
                else:
                    pidx_dict[tok][i] = [j]
    pidx_dict = dict(sorted(pidx_dict.items()))
    positional_index = {}
    terms = pidx_dict.keys()
    for t in terms:
        positional_index[t] = []
        for docID in pidx_dict[t].keys():
            term_doc_positions = copy.deepcopy(pidx_dict[t][docID])
            term_doc_positions.sort()
            positional_index[t].append([docID, term_doc_positions])        
        positional_index[t].sort(key=lambda x: x[0])
    return positional_index


In [47]:
positional_index = create_positional_index(file_toks)

In [48]:
def check_all_in_range(ptrs, ranges):
    for i in range(len(ptrs)):
        if(ptrs[i] >= ranges[i]):
            return False
    return True
# def process_phrase_query(positional_index, query_toks):

#     n_toks = len(query_toks)
#     outer_pointers = [0] * n_toks
#     outer_ranges = [len(positional_index[query_toks[i]]) for i in range(n_toks)]
#     while(check_all_in_range(outer_pointers, outer_ranges)):
#         all_equal = True
#         for i in range(n_toks - 1):
#             if(positional_index[query_toks[i]][outer_pointers[i]][0] != positional_index[query_toks[i + 1]][outer_pointers[i + 1]][0]):
#                 all_equal = False
#                 break
#         if(all_equal == True):
#             inner_pointers = [0] * n_toks
#             inner_ranges = [len(positional_index[query_toks[i]][outer_pointers[i]][1]) for i in range(n_toks)]
#             while(check_all_in_range(inner_pointers, inner_ranges)):
#                 for i in range(n_toks - 1):
                    
def process_phrase_query(positional_index, query_text):

    query_toks = cleanQuery(query_text)
    n_toks = len(query_toks)
    # print(query_toks)
    outer_pointers = [0] * n_toks
    # print(outer_pointers)
    outer_ranges = [len(positional_index[query_toks[i]]) for i in range(n_toks)]
    answer = []
    while(check_all_in_range(outer_pointers, outer_ranges)):
        all_equal = True
        for i in range(n_toks - 1):
            if(positional_index[query_toks[i]][outer_pointers[i]][0] != positional_index[query_toks[i + 1]][outer_pointers[i + 1]][0]):
                all_equal = False
                break
        if(all_equal == False):
            least_ptr = 0
            for i in range(1, n_toks):
                if(positional_index[query_toks[least_ptr]][outer_pointers[least_ptr]][0] > positional_index[query_toks[i]][outer_pointers[i]][0]):
                    least_ptr = i
            outer_pointers[least_ptr] += 1
        if(all_equal == True):

            doc_ID = positional_index[query_toks[0]][outer_pointers[0]][0]
            # print(f"Checking for docid : {doc_ID}")
            posting_lists = []
            for i in range(n_toks):
                posting_lists.append(positional_index[query_toks[i]][outer_pointers[i]][1])
            # print(posting_lists)
            j = 1
            inner_pointers = [0] * n_toks
            inner_ranges = [len(posting_lists[i]) for i in range(n_toks)]
            # print(f"Initial Inner pointers : {inner_pointers}")
            # print(f"Initial Inner ranges : {inner_ranges}\n-------------------\n")
            flag = True
            while(flag == True and check_all_in_range(inner_pointers, inner_ranges)):
                # print(f"Current inner pointers : {inner_pointers}")
                cntr = 0
                for i in range(1, n_toks):
                    if(posting_lists[i][inner_pointers[i]] == posting_lists[i - 1][inner_pointers[i - 1]] + 1):
                        cntr += 1
                
                
                if(cntr == n_toks - 1):
                    # print(f"--M--\nFound a match using innerPointers : {inner_pointers}\n--MM--")
                    flag = False
                    answer.append(doc_ID)
                    break
                else:
                    while(inner_pointers[j] < inner_ranges[j] and (posting_lists[j][inner_pointers[j]] <= posting_lists[j - 1][inner_pointers[j - 1]])):
                        print(j)
                        inner_pointers[j] += 1
                    if(inner_pointers[j] < inner_ranges[j]):
                        
                        if(posting_lists[j][inner_pointers[j]] == posting_lists[j - 1][inner_pointers[j - 1]] + 1):
                            j += 1
                        else:
                            j = 1
                            inner_pointers[0] += 1
            # print("\n-------------------\n")
            # print(answer)

            # doc = positional_index[query_toks[0]][outer_pointers[0]][0]
            # j = 1
            # flag = True
            # inner_pointers = [0] * n_toks
            # inner_ranges = [len(positional_index[query_toks[i]][outer_pointers[i]][1]) for i in range(n_toks)]
            # print(inner_ranges)
            # # print(inner_ranges)
            # while(flag == True and check_all_in_range(inner_pointers, inner_ranges)):
            #     print(inner_pointers)
                
            #     cntr = 0
            #     for i in range(1, n_toks):
            #         # print(positional_index[query_toks[i]][outer_pointers[i]][1][inner_pointers[i]])
            #         if(positional_index[query_toks[i]][outer_pointers[i]][1][inner_pointers[i]] == positional_index[query_toks[i - 1]][outer_pointers[i - 1]][1][inner_pointers[i - 1]] + 1):
            #             cntr += 1
            #     if(cntr == n_toks - 1):
            #         flag = False
            #         answer.append(positional_index[query_toks[0]][outer_pointers[0]][0])
            #         break
     
            #     while((positional_index[query_toks[j]][outer_pointers[j]][1][inner_pointers[j]] <= positional_index[query_toks[j - 1]][outer_pointers[j - 1]][1][inner_pointers[j - 1]]) and inner_pointers[j] < inner_ranges[j]):
            #         inner_pointers[j] += 1
            #     if(inner_pointers[j] < inner_ranges[j]):
            #         if(positional_index[query_toks[j]][outer_pointers[j]][1][inner_pointers[j]] == positional_index[query_toks[j - 1]][outer_pointers[j - 1]][1][inner_pointers[j - 1]] + 1):
            #             j += 1
            #         else:
            #             j = 1
            #             inner_pointers[0] += 1
                
            for i in range(n_toks):
                outer_pointers[i] += 1
    print(answer)
    doc_names = getDocsFromID(file_names, answer)
    print(doc_names)


In [49]:
positional_index['constitutional']

[[21, [85]], [595, [973]], [623, [201]], [942, [168, 174, 4295]]]

In [50]:
positional_index['impediment']

[[21, [86]], [579, [922]]]

In [51]:
# human mechanism so completely between
x = process_phrase_query(positional_index, "human mechanism so completely between")

1
1
1
1
1
1
[122, 334, 763]
['abbott.txt', 'wrdnws2.txt', 'quotes.txt']


In [52]:
print('between' in stopwords.words('english'))

True
